In [1]:
#For data files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#Also clone final project repo
!git clone https://github.com/huggingface/pytorch-pretrained-BERT/

Cloning into 'cis700project'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 68 (delta 33), reused 53 (delta 22), pack-reused 0
Unpacking objects: 100% (68/68), done.
Cloning into 'pytorch-pretrained-BERT'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 3120 (delta 0), reused 0 (delta 0), pack-reused 3119
Receiving objects: 100% (3120/3120), 1.39 MiB | 10.32 MiB/s, done.
Resolving deltas: 100% (2145/2145), done.


In [0]:
#Tensorboard

LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

!if [ -f ngrok ] ; then echo "Ngrok already installed" ; else wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1 && unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1 ; fi

In [4]:
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard Link: ' +str(json.load(sys.stdin)['tunnels'][0]['public_url']))"

Tensorboard Link: https://de400e78.ngrok.io


In [0]:
# Code referenced from https://gist.github.com/gyglim/1f8dfb1b5c82627ae3efcfbbadb9f514
import tensorflow as tf
import numpy as np
import scipy.misc 
try:
    from StringIO import StringIO  # Python 2.7
except ImportError:
    from io import BytesIO         # Python 3.x


class Logger(object):
    
    def __init__(self, log_dir):
        """Create a summary writer logging to log_dir."""
        self.writer = tf.summary.FileWriter(log_dir)

    def scalar_summary(self, tag, value, step):
        """Log a scalar variable."""
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value)])
        self.writer.add_summary(summary, step)

    def image_summary(self, tag, images, step):
        """Log a list of images."""

        img_summaries = []
        for i, img in enumerate(images):
            # Write the image to a string
            try:
                s = StringIO()
            except:
                s = BytesIO()
            scipy.misc.toimage(img).save(s, format="png")

            # Create an Image object
            img_sum = tf.Summary.Image(encoded_image_string=s.getvalue(),
                                       height=img.shape[0],
                                       width=img.shape[1])
            # Create a Summary value
            img_summaries.append(tf.Summary.Value(tag='%s/%d' % (tag, i), image=img_sum))

        # Create and write Summary
        summary = tf.Summary(value=img_summaries)
        self.writer.add_summary(summary, step)
        
    def histo_summary(self, tag, values, step, bins=1000):
        """Log a histogram of the tensor of values."""

        # Create a histogram using numpy
        counts, bin_edges = np.histogram(values, bins=bins)

        # Fill the fields of the histogram proto
        hist = tf.HistogramProto()
        hist.min = float(np.min(values))
        hist.max = float(np.max(values))
        hist.num = int(np.prod(values.shape))
        hist.sum = float(np.sum(values))
        hist.sum_squares = float(np.sum(values**2))

        # Drop the start of the first bin
        bin_edges = bin_edges[1:]

        # Add bin edges and counts
        for edge in bin_edges:
            hist.bucket_limit.append(edge)
        for c in counts:
            hist.bucket.append(c)

        # Create and write Summary
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, histo=hist)])
        self.writer.add_summary(summary, step)
        self.writer.flush()


In [6]:
## Required packages
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
  
import torch
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

    100% |████████████████████████████████| 614.8MB 24kB/s 
  Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.post2
cuda:0


In [7]:
subcategory_size = 0
subcategory2id = {}
with open('/content/drive/My Drive/supercatstats.csv', 'r') as f:
  # build stats
  for index,line in enumerate(f):
    subcategory2id[line.split(',')[0]]=subcategory_size
    subcategory_size += 1
    
print(subcategory_size)

category_size = 0
category2id = {}
with open('/content/drive/My Drive/catstats.csv', 'r') as f:
  # build stats
  for index,line in enumerate(f):
    category2id[line.split(',')[0]]=category_size
    category_size += 1
    
print(category_size)

180
370


In [0]:
import torch
import linecache
import torch.utils.data as data
import re
import sys
sys.path.insert(0, './cis700project')
sys.path.insert(0, './pytorch-pretrained-BERT')
from cis700 import tokenizer
from pytorch_pretrained_bert.tokenization import BertTokenizer

def build_tokenizer():
    tokenizer = BertTokenizer('./cis700project/cis700/vocab/bert-base-uncased-vocab.txt')
    return tokenizer

tok = build_tokenizer()

class AbstractDataset(data.Dataset):
  """Abstract dataset."""

  def __init__(self, file = "joinedlonabstract_en.nt", root_dir = "/content/drive/My Drive"):
    """
    Args:
      csv_file (string): Path to the nt file.
      root_dir (string): Directory with all the images.
    """
    temp = linecache.getline(root_dir + '/' + file, 1)
    self.fullfile = open(root_dir + '/' + file).readlines()
    self.fullfile_len = len(self.fullfile)
    self.root_dir = root_dir
    self.category2id = category2id
    self.subcategory2id = subcategory2id

  def __len__(self):
    return self.fullfile_len

  def __getitem__(self, index):
    #line = linecache.getline(self.root_dir + '/' + self.file, index + 1)
    line = self.fullfile[index]
    match = re.findall(r'(<http://dbpedia.org/[^<]+)', line)
    category = match[2].replace('<http://dbpedia.org/resource/Category:','').replace('>','').replace('\n','').strip()
    subcategory = match[3].replace('<http://dbpedia.org/resource/Category:','').replace('>','').replace('\n','').strip()
    abstract = match[1].replace('<http://dbpedia.org/property/abstract> "','').replace('"@en .',"").replace('\n','').strip()
    tokens = tok.tokenize(abstract)
    if len(tokens) > 256:
      tokens = tokens[:256]
    ids = tok.convert_tokens_to_ids(tokens)
    #print(ids)
    #print(self.category2id[category])
    return torch.Tensor(ids), torch.tensor(self.subcategory2id[subcategory]), torch.tensor(self.category2id[category]) 

  
def collate_fn(data): 
    #Adapted from https://github.com/yunjey/seq2seq-dataloader/blob/master/data_loader.py
    """Creates mini-batch tensors from the list of tuples (src_seq, labels).
    We should build a custom collate_fn rather than using default collate_fn,
    because merging sequences (including padding) is not supported in default.
    Seqeuences are padded to the maximum length of mini-batch sequences (dynamic padding).
    Args:
        data: list of tuple (src_seq, labels).
            - src_seq: torch tensor of shape (?); variable length.
            - subcategory: non-one-hot encoded labels
            - category
    Returns:
        src_seqs: torch tensor of shape (batch_size, padded_length).
        src_lengths: list of length (batch_size); valid length for each padded source sequence.
    """
    def merge(sequences):
        lengths = [len(seq) for seq in sequences]
        padded_seqs = torch.zeros(len(sequences), max(lengths)).long()
        for i, seq in enumerate(sequences):
            end = lengths[i]
            padded_seqs[i, :end] = seq[:end]
        return padded_seqs, lengths

    # sort a list by sequence length (descending order) to use pack_padded_sequence
    data.sort(key=lambda x: len(x[0]), reverse=True)

    # seperate source and target sequences
    src_seqs,subcategory, category = zip(*data)

    # merge sequences (from tuple of 1D tensor to 2D tensor)
    src_seqs, src_lengths = merge(src_seqs)

    return src_seqs, src_lengths,torch.stack(subcategory),torch.stack(category)

In [9]:
import time, datetime
import torch.nn as nn

now = time.mktime(datetime.datetime.now().timetuple())
logger = Logger(f'./logs/biLSTM_{now}/')
logger_val = Logger(f'./logs/biLSTM_eval_{now}/')

#Hyperparameters
batch_size = 512
num_epochs = 8
learning_rate = 3e-3

#Seed
torch.manual_seed(0)

text_dataset = AbstractDataset()
lengths = [int(len(text_dataset)*0.8), int(len(text_dataset)*0.1), int(len(text_dataset)) - 
           int(len(text_dataset)*0.8) - int(len(text_dataset)*0.1)]
text_dataset_train, text_dataset_val, text_dataset_test = torch.utils.data.random_split(text_dataset, lengths)

loader_train = torch.utils.data.DataLoader(text_dataset_train,shuffle=True, batch_size=batch_size, collate_fn=collate_fn,num_workers=3,pin_memory=True)
loader_test = torch.utils.data.DataLoader(text_dataset_test,shuffle=True, batch_size=batch_size, collate_fn=collate_fn,num_workers=3,pin_memory=True)
torch.backends.cudnn.enabled = False

# Bidirectional recurrent neural network (many-to-one)
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embed = nn.Embedding(30522, input_size)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # 2 for bidirection
    
    def forward(self, x):
        # Set initial states
        #h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) # 2 for bidirection 
        #c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        #Embed
        x = self.embed(x)
        
        # Forward propagate LSTM
        x, _ = self.lstm(x)  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        
        # Decode the hidden state of the last time step
        x = self.fc(x[:, -1, :])
        return x

model = BiLSTM(input_size=50, hidden_size=100, num_layers=1, num_classes=category_size).to(device)

#Loss and optimizer
loss_fun = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

def validate(num_train_steps):     
  #Get validation data
  rand_sampler = torch.utils.data.RandomSampler(text_dataset_val, num_samples=batch_size, replacement=True)
  loader_val = torch.utils.data.DataLoader(text_dataset_val, batch_size=batch_size, collate_fn=collate_fn,num_workers=3,pin_memory=True, sampler=rand_sampler)
  get_data_step = iter(loader_val)
  model.eval()
  count_correct = 0
  count_total = 0
  with torch.no_grad():
    abstract, abstract_lengths, subcategory, category = next(get_data_step)

    abstract = abstract.to(device)
    subcategory = subcategory.to(device)
    category = category.to(device)

    result = model(abstract)
    loss = loss_fun(result, category)
    _, argmax = torch.max(result, 1)
    count_total += category.size(0)
    count_correct += (argmax == category).sum().item()
    accuracy = (category == argmax).float().mean()
  
    #Tensorboard logging
    to_log = {'loss': loss.item(), 'accuracy': accuracy.item()}
    for handle, val in to_log.items():
      logger_val.scalar_summary(handle, val, num_train_steps+1)

  print('The validation accuracy is: %s%% [%s]' % (count_correct/count_total * 100,batch_size))  
  model.train()

def evaluate():     
  #Get testing data
  get_data_step = iter(loader_test)
  epoch_length = len(loader_test)

  model.eval()
  count_correct = 0
  count_total = 0
  with torch.no_grad():
    for test_step in range(epoch_length):
      abstract, abstract_lengths, subcategory, category = next(get_data_step)

      abstract = abstract.to(device)
      subcategory = subcategory.to(device)
      category = category.to(device)

      result = model(abstract)
      _, argmax = torch.max(result, 1)
      count_total += category.size(0)
      count_correct += (argmax == category).sum().item()

  print('The testing set accuracy is: %s%% [%s]' % (count_correct/count_total * 100,epoch_length * batch_size))  
  model.train()
  
#Training loop
model.train()
num_train_steps = 0
for epoch in range(num_epochs):
    #Get an epoch
    get_data_step = iter(loader_train)
    epoch_length = len(loader_train)
    for train_step in range(epoch_length):
      #start = time.time()
      num_train_steps += 1
      
      abstract, abstract_lengths, subcategory, category = next(get_data_step)
      #print(abstract.size())
      abstract = abstract.to(device)
      subcategory = subcategory.to(device)
      category = category.to(device)
    
      # Do a forward pass
      #start = time.time()
      result = model(abstract)
      #end = time.time()
      loss = loss_fun(result, category)
    
      # Now backpropagate
      optimizer.zero_grad()
      #start = time.time()
      loss.backward()
      #end = time.time()
      
      optimizer.step()

      # Find the accuracy
      _, argmax = torch.max(result,1)
      accuracy = (category == argmax).float().mean()
      
      #Print
      if (num_train_steps + 1) % 100 == 0: 
        print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f, Accuracy: %4f'
          % (epoch + 1, num_epochs, num_train_steps, len(loader_train) * num_epochs, loss.item(), accuracy.item())) 
        validate(num_train_steps+1)
      #print(torch.cuda.memory_allocated(device))
      #print(torch.cuda.memory_cached(device))
      end = time.time()
      #print(end-start)
      del abstract, abstract_lengths, subcategory, category, result, argmax
      torch.cuda.empty_cache()
      
      if num_train_steps % 1 == 0:  
        #Tensorboard logging
        to_log = {'loss': loss.item(), 'accuracy': accuracy.item()}
        for handle, val in to_log.items():
            logger.scalar_summary(handle, val, num_train_steps+1)
            
    evaluate()

Epoch: [ 1/ 8], Step: [ 99/ 17312], Loss: 5.3816, Accuracy: 0.048828
The validation accuracy is: 6.4453125% [512]
Epoch: [ 1/ 8], Step: [ 199/ 17312], Loss: 5.3469, Accuracy: 0.080078
The validation accuracy is: 5.859375% [512]
Epoch: [ 1/ 8], Step: [ 299/ 17312], Loss: 5.3896, Accuracy: 0.041016
The validation accuracy is: 2.9296875% [512]
Epoch: [ 1/ 8], Step: [ 399/ 17312], Loss: 4.9805, Accuracy: 0.082031
The validation accuracy is: 8.0078125% [512]
Epoch: [ 1/ 8], Step: [ 499/ 17312], Loss: 4.9285, Accuracy: 0.091797
The validation accuracy is: 8.984375% [512]
Epoch: [ 1/ 8], Step: [ 599/ 17312], Loss: 5.1298, Accuracy: 0.070312
The validation accuracy is: 9.5703125% [512]
Epoch: [ 1/ 8], Step: [ 699/ 17312], Loss: 4.6173, Accuracy: 0.121094
The validation accuracy is: 9.765625% [512]
Epoch: [ 1/ 8], Step: [ 799/ 17312], Loss: 4.6023, Accuracy: 0.113281
The validation accuracy is: 11.71875% [512]
Epoch: [ 1/ 8], Step: [ 899/ 17312], Loss: 4.5458, Accuracy: 0.109375
The validation 